# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

---------

## İş Problemi

Gezinomi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı
(level based) yeni satış tanımları oluşturmak ve bu yeni satış
tanımlarına göre segmentler oluşturup bu segmentlere göre yeni 
gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini
tahmin etmek istemektedir. 
Örneğin:
Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen
bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek
isteniyor

Örneğin:
Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen
bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek
isteniyor.

-------

In [2]:
# Pandas kütüphanesini import edelim;
import pandas as pd
import numpy as np

# pandas görüntü ayarlarını yapalım;

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_ = pd.read_excel(r"C:\Users\kkakt\Desktop\case's\dataset\miuul_gezinomi.xlsx")

df = df_.copy()

df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.304,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.971,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.839,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.711,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.476,Yarım Pansiyon,İzmir,Saturday,2,Low


In [93]:
# veri seti için genel bir inceleme yapalım;

from et_analysis import check_df

check_df(df)

##################### Shape #####################
(59164, 10)
##################### Types #####################
SaleId                         int64
SaleDate              datetime64[ns]
CheckInDate           datetime64[ns]
Price                        float64
ConceptName                   object
SaleCityName                  object
CInDay                        object
SaleCheckInDayDiff             int64
Seasons                       object
EB_SCORE                    category
dtype: object
##################### NA #####################
                    # of N/A values
SaleId                            0
SaleDate                          0
CheckInDate                       0
Price                            13
ConceptName                       0
SaleCityName                      0
CInDay                            0
SaleCheckInDayDiff                0
Seasons                           0
EB_SCORE                          0
##################### Quantiles #####################
       

In [3]:
# kaç adet unique konsept olduğunu inceleyelim;

df["ConceptName"].unique()
df["ConceptName"].nunique()


3

In [4]:
# Unique şehirleri ve frekanslarını incelyelim;
df["SaleCityName"].unique()



# oransal dağılımı göstermek için:
#df["SaleCityName"].value_counts(normalize=True)


array(['Antalya', 'İzmir', 'Diğer', 'Aydın', 'Muğla', 'Girne'],
      dtype=object)

In [5]:
# Hangi Concept’den kaçar tane satış gerçekleşmiş?

df["SaleCityName"].value_counts()

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

In [6]:
# Şehirlere göre satışlardan toplam ne kadar kazanıldığını bulalım;

df.groupby("SaleCityName").agg({"Price":"sum"})

,Price
SaleCityName,
Antalya,2041911.096
Aydın,573296.009
Diğer,154572.290
Girne,27065.028
Muğla,665842.213
İzmir,165934.834


In [7]:
# Conceptlere göre satışlardan toplam ne kadar kazanıldığını bulalım;

df.groupby("ConceptName").agg({"Price":"sum"})

,Price
ConceptName,
Herşey Dahil,3332910.768
Oda + Kahvaltı,121308.353
Yarım Pansiyon,174402.350


In [8]:
# Şehirlere göre price ortalmalarını bulalım;


df.groupby("SaleCityName").agg({"Price":"mean"})

,Price
SaleCityName,
Antalya,64.521
Aydın,53.856
Diğer,47.707
Girne,59.484
Muğla,62.462
İzmir,66.268


In [9]:
# Şehirlere göre price ortalmalarını bulalım;

df.groupby("ConceptName").agg({"Price":"mean"})

,Price
ConceptName,
Herşey Dahil,62.672
Oda + Kahvaltı,50.252
Yarım Pansiyon,49.031


In [10]:
# Şehir-Concept kırılımındaPRICE ortalamaları bulalım;

df.groupby(["SaleCityName","ConceptName"]).agg({"Price":"mean"})

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil   64.519
             Oda + Kahvaltı 63.505
             Yarım Pansiyon 67.191
Aydın        Herşey Dahil   53.995
             Oda + Kahvaltı 34.459
             Yarım Pansiyon 30.017
Diğer        Herşey Dahil   84.771
             Oda + Kahvaltı 37.599
             Yarım Pansiyon 42.113
Girne        Herşey Dahil   97.681
             Oda + Kahvaltı 39.776
             Yarım Pansiyon 53.248
Muğla        Herşey Dahil   63.020
             Oda + Kahvaltı 59.038
             Yarım Pansiyon 45.121
İzmir        Herşey Dahil   74.702
             Oda + Kahvaltı 41.320
             Yarım Pansiyon 59.610

In [94]:
# SelesCheckInDayDiff değişkenini kategorik değişkenine çevirelim;

rangeDay = [-1,7,30,90,df["SaleCheckInDayDiff"].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]



In [96]:
df["EB_SCORE"] = pd.cut(x = df["SaleCheckInDayDiff"],bins =rangeDay, labels = labels )
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,EB_SCORE
0,415122,2022-12-03,2022-12-03,79.304,Herşey Dahil,Antalya,Saturday,0,Low,Last Minuters
1,415103,2022-12-03,2022-12-03,45.971,Yarım Pansiyon,Antalya,Saturday,0,Low,Last Minuters
2,404034,2022-09-12,2022-09-13,77.839,Herşey Dahil,Antalya,Tuesday,1,High,Last Minuters
3,415094,2022-12-03,2022-12-10,222.711,Yarım Pansiyon,İzmir,Saturday,7,Low,Last Minuters
4,414951,2022-12-01,2022-12-03,140.476,Yarım Pansiyon,İzmir,Saturday,2,Low,Last Minuters


In [26]:
# Şehirleri konspetlere ve sezonlarına göre fiyat ortalaması ve satış adetini inceleyelim;


agg_df = df.groupby(["SaleCityName","ConceptName","Seasons"]).agg({"Price":["mean","count"]})

In [27]:
# Şehirleri konspetlere ve cinday göre fiyat ortalaması ve satış adetini inceleyelim;


df.groupby(["SaleCityName","ConceptName","CInDay"]).agg({"Price":["mean","count"]})

Price      
                                         mean count
SaleCityName ConceptName    CInDay                 
Antalya      Herşey Dahil   Friday     62.659  4136
                            Monday     63.259  6831
                            Saturday   64.416  4741
                            Sunday     65.849  3818
                            Thursday   62.893  3898
                            Tuesday    66.772  3760
                            Wednesday  67.174  3609
             Oda + Kahvaltı Friday     63.128   114
                            Monday     57.173    66
                            Saturday   58.012   117
                            Sunday     63.469    46
                            Thursday   64.653    93
                            Tuesday    65.836    73
                            Wednesday  72.809    89
             Yarım Pansiyon Friday     69.256    38
                            Monday     76.954    20
                            Saturday   63.375    71
                            Sunday     64.448    30
                            Thursday   66.637    40
                            Tuesday    71.019    21
                            Wednesday  67.779    36
Aydın        Herşey Dahil   Friday     51.034  1825
                            Monday     54.385  2566
                            Saturday   56.887  1702
                            Sunday     53.194  1098
                            Thursday   54.514  1315
                            Tuesday    53.507  1098
                            Wednesday  54.219   972
             Oda + Kahvaltı Friday     31.174     7
                            Monday     45.400     7
                            Saturday   23.931     2
                            Sunday     43.528     3
                            Thursday   37.521     7
                            Tuesday    32.929     3
                            Wednesday  25.949     9
             Yarım Pansiyon Friday     24.516     5
                            Monday     43.430     3
                            Saturday   29.703     8
                            Sunday     30.284     3
                            Thursday   22.778     5
                            Tuesday    32.160     4
                            Wednesday  35.546     3
Diğer        Herşey Dahil   Friday     84.657    78
                            Monday     90.598   112
                            Saturday   77.226    90
                            Sunday     81.613    57
                            Thursday   76.109    54
                            Tuesday    90.329    56
                            Wednesday  91.183    58
             Oda + Kahvaltı Friday     38.466   116
                            Monday     36.649    87
                            Saturday   36.401   207
                            Sunday     38.064    67
                            Thursday   35.394    99
                            Tuesday    42.241    80
                            Wednesday  38.052   101
             Yarım Pansiyon Friday     42.374   407
                            Monday     40.278   217
                            Saturday   43.247   638
                            Sunday     41.861   219
                            Thursday   41.945   198
                            Tuesday    43.502   140
                            Wednesday  38.739   159
Girne        Herşey Dahil   Friday    121.194    12
                            Monday     98.662    22
                            Saturday  109.165     9
                            Sunday     85.192    38
                            Thursday  112.379     7
                            Tuesday   111.397     8
                            Wednesday  83.168    10
             Oda + Kahvaltı Friday     45.024    34
                            Monday     40.763    13
                            Saturday   37.175    29
                            Sunday     38.216    20
                            Thursday

In [43]:
# Şehir konsept ve sezon kırılımına fiyata göre sıralayalım;

agg_df = df.groupby(["SaleCityName","ConceptName","Seasons"]).agg({"Price":"mean"})

agg_df = agg_df.sort_values("Price",ascending=False)

agg_df.head()

Price
SaleCityName ConceptName    Seasons        
Girne        Herşey Dahil   High    103.935
                            Low      90.936
İzmir        Yarım Pansiyon High     87.657
Diğer        Herşey Dahil   Low      87.311
                            High     83.787

In [44]:
# indekste yer alan çıktıları kendi isimlerine göre çevirelim;

agg_df= agg_df.reset_index()
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935
1,Girne,Herşey Dahil,Low,90.936
2,İzmir,Yarım Pansiyon,High,87.657
3,Diğer,Herşey Dahil,Low,87.311
4,Diğer,Herşey Dahil,High,83.787


In [77]:
# seviye tanımlı satışı tanımlayalım; 

agg_df["sales_level_based"] = agg_df["SaleCityName"].str.upper() + "_" + agg_df["ConceptName"].str.upper() + "_" + agg_df["Seasons"].str.upper()
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.935,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.936,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.657,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.311,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.787,DIĞER_HERŞEY DAHIL_HIGH


In [97]:
# diğer bir tanımlama örneği olarak;

agg_df[["sales_level_based"]].agg(lambda x: "_".join(x).upper(),axis=1)

0         GIRNE_HERŞEY DAHIL_HIGH
1          GIRNE_HERŞEY DAHIL_LOW
2       İZMIR_YARIM PANSIYON_HIGH
3          DIĞER_HERŞEY DAHIL_LOW
4         DIĞER_HERŞEY DAHIL_HIGH
5         İZMIR_HERŞEY DAHIL_HIGH
6          İZMIR_HERŞEY DAHIL_LOW
7     ANTALYA_YARIM PANSIYON_HIGH
8     ANTALYA_ODA + KAHVALTI_HIGH
9       ANTALYA_HERŞEY DAHIL_HIGH
10        MUĞLA_HERŞEY DAHIL_HIGH
11     ANTALYA_YARIM PANSIYON_LOW
12       ANTALYA_HERŞEY DAHIL_LOW
13      MUĞLA_ODA + KAHVALTI_HIGH
14     ANTALYA_ODA + KAHVALTI_LOW
15      GIRNE_YARIM PANSIYON_HIGH
16       İZMIR_YARIM PANSIYON_LOW
17        AYDIN_HERŞEY DAHIL_HIGH
18      İZMIR_ODA + KAHVALTI_HIGH
19       GIRNE_YARIM PANSIYON_LOW
20         MUĞLA_HERŞEY DAHIL_LOW
21      MUĞLA_YARIM PANSIYON_HIGH
22       AYDIN_ODA + KAHVALTI_LOW
23       DIĞER_YARIM PANSIYON_LOW
24       GIRNE_ODA + KAHVALTI_LOW
25       DIĞER_ODA + KAHVALTI_LOW
26      DIĞER_YARIM PANSIYON_HIGH
27      GIRNE_ODA + KAHVALTI_HIGH
28       MUĞLA_ODA + KAHVALTI_LOW
29      DIĞER_

In [80]:
# oluşturduğumuz personoları segmentlere göre ayıralım;
agg_df["segment"] = pd.qcut(agg_df["Price"], 4 , ["D","C","B","A"])

agg_df.tail()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,segment
31,İzmir,Oda + Kahvaltı,Low,33.555,İZMIR_ODA + KAHVALTI_LOW,D
32,Muğla,Yarım Pansiyon,Low,32.682,MUĞLA_YARIM PANSIYON_LOW,D
33,Aydın,Yarım Pansiyon,High,32.626,AYDIN_YARIM PANSIYON_HIGH,D
34,Aydın,Oda + Kahvaltı,High,30.388,AYDIN_ODA + KAHVALTI_HIGH,D
35,Aydın,Yarım Pansiyon,Low,25.272,AYDIN_YARIM PANSIYON_LOW,D


In [100]:
# segmentlere göre yorumlama yapalım;

# Case1 : Antalya'da her şey dahil ve yüksek sezonda tatil yapmak isteyen
# bir kişinin ortalama ne kadar gelir getirmesi beklenir.


new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"]==new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,segment
9,Antalya,Herşey Dahil,High,64.920,ANTALYA_HERŞEY DAHIL_HIGH,B


In [91]:
# Case2 : Girne'de yarım pansiyon bir otele düşük sezonda giden tatilci hangi segmentte yer alacaktır.

agg_df.loc [(agg_df["SaleCityName"] == "Girne") & (agg_df["ConceptName"] == "Yarım Pansiyon")&(agg_df["Seasons"] == "Low"),"segment"]

19    C
Name: segment, dtype: category
Categories (4, object): ['D' < 'C' < 'B' < 'A']